In [1]:
import mysql.connector
import pymysql
import os
import subprocess
import time

def export_remote_mysql_database(remote_host, remote_user, remote_password, remote_database, output_dir):
    """
    Exports a full MySQL database from a remote server using mysqldump.

    Args:
        remote_host (str): Remote MySQL host.
        remote_user (str): Remote MySQL user.
        remote_password (str): Remote MySQL password for the given user.
        remote_database (str): Name of the remote database to be exported.
        output_dir (str): Directory where the export file will be saved.

    Returns:
        str: The filename of the exported backup file.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Generate the export filename based on the current timestamp
    export_filename = f"{remote_database}_{time.strftime('%Y-%m-%d_%H-%M-%S')}.sql"
    export_path = os.path.join(output_dir, export_filename)

    print(f"Starting database export to: {export_path}")

    # Construct the mysqldump command for remote export
    command = f"mysqldump --host={remote_host} --user={remote_user} --password={remote_password} --default-character-set=utf8mb4 -N --routines --skip-triggers  {remote_database} > {export_path}"

    try:
        # Execute the mysqldump command for remote export
        subprocess.run(command, shell=True, check=True)
        print(f"Database '{remote_database}' exported successfully. Export file: {export_path}")
        return export_path
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while exporting the remote database: {e}")
        return None

def import_to_azure_mariadb(azure_host, azure_user, azure_password, azure_database, import_file, connection):
    """
    Imports a MySQL database backup file to Azure MariaDB.

    Args:
        azure_host (str): Azure MariaDB host.
        azure_user (str): Azure MariaDB user.
        azure_password (str): Azure MariaDB password for the given user.
        azure_database (str): Name of the database to import the backup into.
        import_file (str): Path to the MySQL database backup file.

    Returns:
        bool: True if the import was successful, False otherwise.
    """
    # Construct the mysql command for importing the backup into Azure MariaDB
    command = f"mysql --host={azure_host} --user={azure_user} --password={azure_password} --ssl-mode=VERIFY_CA --ssl-ca=.\DigiCertGlobalRootG2.crt.pem {azure_database} < {import_file}"
    # command = f"'mysql --host={azure_host} --user={azure_user} --password={azure_password} --default-character-set=utf8mb4 -N --routines --skip-triggers {azure_database}  < {import_file}"
    try:
        # Check if the database exists on the cloud DB
        db_exists = mysql_database_exists(connection = connection,database_name = azure_database)
        if db_exists:
            # Execute the mysql command to import the backup into Azure MariaDB
            subprocess.run(command, shell=True, check=True)
            print(f"Backup successfully imported into '{azure_database}' in Azure MariaDB.")
            return True
        else:
            create_mysql_database(connection = connection,database_name = azure_database)
            subprocess.run(command, shell=True, check=True)
            print(f"Backup successfully imported into '{azure_database}' in Azure MariaDB.")
            return True
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while importing the backup: {e}")
        return False

# Function to check if a database exists in MySQL
def mysql_database_exists(connection, database_name):
    try:
        with connection.cursor() as cursor:
            cursor.execute("SHOW DATABASES")
            databases = [row[0] for row in cursor]
            return database_name in databases
    except mysql.connector.Error as err:
        print(f"Error checking if database exists: {err}")
        return False

# Function to create a database in MySQL
def create_mysql_database(connection, database_name):
    try:
        with connection.cursor() as cursor:
            cursor.execute(f"CREATE DATABASE {database_name}")
        print(f"Database '{database_name}' created successfully!")
    except mysql.connector.Error as err:
        print(f"Error creating database: {err}")

# Function to connect to the source MySQL database
def connect_to_mysql(host, user, password, database):
    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            # database=database
        )
        print("Connected to MySQL successfully!")
        return conn
    except mysql.connector.Error as err:
        print(f"Error connecting to MySQL: {err}")
        return None

# Function to connect to the destination MariaDB database on Azure
def connect_to_mariadb_azure(host, user, password, database):
    try:
        conn = pymysql.connect(
            host=host,
            user=user,
            password=password,
            # database=database
        )
        print("Connected to MariaDB on Azure successfully!")
        return conn
    except pymysql.Error as err:
        print(f"Error connecting to MariaDB on Azure: {err}")
        return None



In [2]:
# Replace these with your remote MySQL credentials
remote_mysql_host = "10.30.42.24"
remote_mysql_user = "root"
remote_mysql_password = ""
remote_mysql_database = "escwabeta"
export_directory = "./db_exports/"

# Replace these with your Azure MariaDB credentials
azure_host = "drupal-d-mariadb-001.mariadb.database.azure.com"
azure_user = "drupaladmin@drupal-d-mariadb-001.mariadb.database.azure.com"
azure_password = "UNescwa2020@@"
azure_database = "escwabeta"

"""

cmd /c  "mysql --host={azure_host} \
    --user={remote_mysql_user} \
    --password={remote_mysql_password}\
    --ssl-mode=VERIFY_CA\
    --ssl-ca=".\DigiCertGlobalRootG2.crt.pem"\
    {azure_database} < {import_file}  >output.log"
"""

# Initializing the connections 
remote_db = connect_to_mysql(host = remote_mysql_host, user = remote_mysql_user, password = remote_mysql_password, database = remote_mysql_database )
cloud_db = connect_to_mariadb_azure(host = azure_host, user = azure_user, password = azure_password, database = azure_database)

# Exporting the on prem's DB
db_path = export_remote_mysql_database(remote_mysql_host, remote_mysql_user, remote_mysql_password, remote_mysql_database, export_directory)

# Importing the database into mariaDb
import_to_azure_mariadb(azure_host = azure_host, azure_user = azure_user, azure_password = azure_password, azure_database = azure_database,connection = cloud_db, import_file = db_path)


Connected to MySQL successfully!
Connected to MariaDB on Azure successfully!
Database 'escwabeta' exported successfully. Export file: ./db_exports/escwabeta_2023-07-27_12-54-46.sql
Database 'escwabeta' created successfully!
Backup successfully imported into 'escwabeta' in Azure MariaDB.


True

In [21]:
# restore command
cat = subprocess.run("mysqldump --host=10.30.42.24 --user=root --password= --default-character-set=utf8mb4 -N --routines --skip-triggers  escwabeta > ./escwabeta_2023-07-26_14-03-31.sql", shell=True, check=True, stdout=subprocess.PIPE)

Backup successfully imported into 'escwabeta' in Azure MariaDB.


True

In [23]:

# subprocess.run(f'mysql --host={azure_host} --user={azure_user} --password={azure_password} --default-character-set=utf8mb4 -N --routines --skip-triggers {azure_database}  < escwabeta_2023-07-26_14-03-31.sql')
f'mysql --host={azure_host} --user={azure_user} --password={azure_password} --default-character-set=utf8mb4 -N --routines --skip-triggers {azure_database}  < escwabeta_2023-07-26_14-03-31.sql'

'mysql --host=drupal-d-mariadb-001.mariadb.database.azure.com --user=drupaladmin@drupal-d-mariadb-001.mariadb.database.azure.com --password=UNescwa2020@@ --default-character-set=utf8mb4 -N --routines --skip-triggers escwabeta  < escwabeta_2023-07-26_14-03-31.sql'

In [13]:
remote_mysql_host = "10.30.42.24"
remote_mysql_user = "root"
remote_mysql_password = ""
remote_mysql_database = "escwabeta"
export_directory = "/escwabeta/"

os.system(f'mysqldump --host={remote_mysql_host} --user={remote_mysql_user} --password={remote_mysql_password} {remote_mysql_database}  > database.sql')


2

In [19]:
f'mysqldump --host={remote_mysql_host} --user={remote_mysql_user} --password={remote_mysql_password} {remote_mysql_database}  > database.sql'

'mysqldump --host=10.30.42.24 --user=root --password= escwabeta  > database.sql'

In [5]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

def transfer_files_to_azure_blob_storage(local_folder_path, storage_account_name, container_name):
    try:
        # Replace the following variables with your Azure Blob Storage connection string and the container name
        connection_string = "DefaultEndpointsProtocol=https;AccountName=drupalstoragev1;AccountKey=fWVfket0+ldSDI31pHFICYVPbb7ZUFHAdk226aIPQp42jlzgznVJgQ/mRAx7hUDx3/JT9KhpTfWJ+AStw7dK6w==;EndpointSuffix=core.windows.net"

        # Create a blob service client
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)

        # Create a container client if it doesn't exist
        container_client = blob_service_client.get_container_client(container_name)
        if not container_client.exists():
            container_client.create_container()

        # Get a list of all files in the local folder
        local_files = os.listdir(local_folder_path)

        # Transfer each file to the Azure Blob Storage container
        for file_name in local_files:
            local_file_path = os.path.join(local_folder_path, file_name)

            # Create a blob client for the file
            blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

            # Upload the file to the blob storage
            with open(local_file_path, "rb") as data:
                blob_client.upload_blob(data, overwrite=True)

            print(f"File '{file_name}' has been transferred to Azure Blob Storage.")

    except Exception as e:
        print("An error occurred:", e)

# Example usage:
local_folder = "../infrastructure"  # Replace this with the path to your local 'files' folder
storage_account = "drupalstoragev1"
container = "escwa-corporate"

transfer_files_to_azure_blob_storage(local_folder, storage_account, container)


File 'alp.bicep' has been transferred to Azure Blob Storage.
File 'main.tf' has been transferred to Azure Blob Storage.


In [17]:
def read_file_by_lines(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
        return lines
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return []


def save_file_with_lines( new_file_path, lines):
    try:
        with open(new_file_path, 'w') as new_file:
            new_file.writelines(lines)
        print(f"File has been saved to: {new_file_path}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")

In [21]:
file_path = "settings_model.php"  # Replace this with the path of the file you want to read
lines = read_file_by_lines(file_path)
if lines:
    for line in lines:
        print(line.strip())  # Using strip() to remove newline characters from each line
else:
    print("File is empty or could not be read.")

<?php

// @codingStandardsIgnoreFile

/**
* @file
* Drupal site-specific configuration file.
*
* IMPORTANT NOTE:
* This file may have been set to read-only by the Drupal installation program.
* If you make changes to this file, be sure to protect it again after making
* your modifications. Failure to remove write permissions to this file is a
* security risk.
*
* In order to use the selection rules below the multisite aliasing file named
* sites/sites.php must be present. Its optional settings will be loaded, and
* the aliases in the array $sites will override the default directory rules
* below. See sites/example.sites.php for more information about aliases.
*
* The configuration directory will be discovered by stripping the website's
* hostname from left to right and pathname from right to left. The first
* configuration file found will be used and any others will be ignored. If no
* other configuration file is found then the default configuration file at
* 'sites/default' will be us

In [22]:
azure_host = "drupal-d-mariadb-001.mariadb.database.azure.com"
azure_user = "drupaladmin@drupal-d-mariadb-001.mariadb.database.azure.com"
azure_password = "UNescwa2020@@"
azure_database = "escwabeta"
application_DNS = "beta-dev.unescwa"

updated_config = f"$databases['default']['default'] = array (\
  'database' => '{azure_database}',\
  'username' => '{azure_user}',\
  'password' => '{azure_password}',\
  'prefix' => '',\
  'host' => '{azure_host}',\
  'port' => '3306',\
  'namespace' => 'Drupal\\Core\\Database\\Driver\\mysql',\
  'driver' => 'mysql',\
  'options' => array( PDO::MYSQL_ATTR_SSL_CA => './DigiCertGlobalRootG2.crt.pem')\
);\
$settings['trusted_host_patterns'] = array(\
    '^{application_DNS}\.org$',\
    '^.+\{application_DNS}\.org$',\
    );\
"



In [24]:
file_path = "model_settings.php"  # Replace this with the path of the file you want to read
lines = read_file_by_lines(file_path)

azure_host = "drupal-d-mariadb-001.mariadb.database.azure.com"
azure_user = "drupaladmin@drupal-d-mariadb-001.mariadb.database.azure.com"
azure_password = "UNescwa2020@@"
azure_database = "escwabeta"
application_DNS = "beta-dev.unescwa"

updated_config = f"$databases['default']['default'] = array (\
'database' => '{azure_database}',\
'username' => '{azure_user}',\
'password' => '{azure_password}',\
'prefix' => '',\
'host' => '{azure_host}',\
'port' => '3306',\
'namespace' => 'Drupal\\Core\\Database\\Driver\\mysql',\
'driver' => 'mysql',\
'options' => array( PDO::MYSQL_ATTR_SSL_CA => './DigiCertGlobalRootG2.crt.pem')\
);\
$settings['trusted_host_patterns'] = array(\
    '^{application_DNS}\.org$',\
    '^.+\{application_DNS}\.org$',\
    );\
"

lines[800] = updated_config
del lines[801:815]
save_file_with_lines( new_file_path='./settings_updates.php', lines= lines)




File has been saved to: ./settings_updates.php


In [12]:
with open ('./Dockerfile','r') as f:
    file = f.readlines()


In [13]:
file

['FROM php:8.1-apache-bullseye\n',
 '# Copy the Code folder to the container\n',
 'COPY ./ /var/www/html/<PROJECT-NAME>/\n',
 'COPY ./apache-config/ /etc/apache2/sites-available/\n',
 'RUN ln -s /etc/apache2/sites-available/<PROJECT-NAME>.conf /etc/apache2/sites-enabled/\n',
 'EXPOSE 80\n',
 'EXPOSE 443\n',
 '\n',
 '\n',
 '# Debian Buster configuration\n',
 'RUN apt update -y --fix-missing\n',
 'RUN apt upgrade -y\n',
 'RUN apt install -y apt-utils nano wget dialog software-properties-common build-essential git curl openssl\n',
 '\n',
 '# PHP Module: zip\n',
 'RUN apt install -y libzip-dev unzip\n',
 'RUN docker-php-ext-install zip\n',
 '\n',
 '# PHP Module: intl\n',
 'RUN apt install -y libicu-dev\n',
 'RUN docker-php-ext-install -j$(nproc) intl\n',
 '\n',
 '# PHP Module: gd\n',
 'RUN apt install -y libfreetype6-dev libjpeg62-turbo-dev libpng-dev\n',
 'RUN docker-php-ext-install -j$(nproc) gd\n',
 '\n',
 '# PHP Module: bcmath\n',
 'RUN docker-php-ext-install bcmath\n',
 '\n',
 '# PHP 